In [1]:
import sys
from demoparser.demofile import DemoFile
import matplotlib.pyplot as plt
import time

#Allow graphs to open in a window instead of inline
%matplotlib qt

In [ ]:
#Global Variables
g_map_xrange = range()
g_map_yrange = range()

In [2]:
#Encapsulates all information about each individual player
class Person:
    def __init__ (self, name, team):
        self.m_name = name
        self.m_team = team
        self.m_position_history = {
                                    'dead' : [],
                                    'position': [],
                                    'view_angle' : [],
                                    'spotted' : []
                                    }
        self.m_dead = 'N'
        
    def update_position (self, pos, angle, spotted, dead):
        #current position is now at m_position_history[-1]
        if self.m_dead == 'Y': return
        if dead == 'Y': self.m_dead = 'Y'
        
        self.m_position_history['position'].append(pos)
        self.m_position_history['view_angle'].append(angle)
        self.m_position_history['dead'].append(dead)
        self.m_position_history['spotted'].append(spotted)
        
        

In [3]:
#Encapsulates parsed information from DEM file about each round, no interesting processing is conducted here.
class Round:
    def __init__ (self,rid):
        self.m_rid = rid
        self.m_players = dict()
        self.m_number_frames = 0
    
    
    def add_player (self,player):
        #Add person type to list of players in round
        self.m_players[player.m_name] = player
    
    
    def update_player_position (self, name, posX, posY, ang, spotted,dead):
        #Despite name, updates position, view ang (does not work), deadiness and wether they are visible or not
        self.m_players[name].update_position([posX, posY], ang, spotted, dead)
    
    
    def print_round (self):
        #relised after implementation that I should have used a scatter graph for this... 
        plt.ion()
        figure = plt.figure()
        plt.axis([-2600, 2100, -1200, 3200])
        plot_list = []
        max_frames = 0
        img = plt.imread("de_dust_map.png")
        plt.imshow(img, extent=[-2600,2100,-1200,3200])
        
        for key, play in self.m_players.items():
            if play.m_team == "CT":
                o, = plt.plot([],'bo',markersize=13, markeredgewidth=3)
            elif play.m_team == "TERRORIST":
                o, = plt.plot([],'ro',markersize=13, markeredgewidth=3)
            plot_list.append(o)
            
            if max_frames < len(play.m_position_history['position']):
                max_frames = len(play.m_position_history['position'])
        
        plt.show()  
        
        for frame_id in range(max_frames):
            self.draw_frame(frame_id, plot_list, figure)
            #Wait for one tick (think it is 60 ticks per second), changed so i dont have to wait forever to see results
            #Not entirely sure on tick to seconds conversion...TODO
            time.sleep(1/60 * 1/12)
            
            
    def draw_frame(self, fid, plots, fig):
        count=0
        
        for key, play in self.m_players.items():
            if not play.m_position_history['position']: continue
            try:
                if not play.m_position_history['position'][fid]: continue
                
                plots[count].set_xdata(play.m_position_history['position'][fid][0])
                plots[count].set_ydata(play.m_position_history['position'][fid][1])
                
                if play.m_position_history['dead'][fid] == 'Y':
                    plots[count].set_markeredgecolor('k')
                    plots[count].set_marker('x')
                
                else:
                    plots[count].set_marker('o')
                    
                    if play.m_position_history['spotted'][fid] == True:
                        plots[count].set_markeredgecolor('k')
                    
                    else:
                        plots[count].set_markeredgecolor('w')
                
            except:
                pass
            
            count+=1
            fig.canvas.draw()
            fig.canvas.flush_events()

In [4]:
current_round = [0]

#Callback Functions
def tick_start(msg):
    #If first round has not begun yet, return
    if m_rounds == []: return

    for play in d.entities.players:      
        if not play.team: continue
        if play.team.name == "Spectator": continue
        spotted = play.get_prop('DT_BaseEntity', 'm_bSpotted')
        m_rounds[-1].update_player_position(play.name,int(play.position['x']), int(play.position['y']), play.view_angle, spotted,'N')
            
def round_start (msg, msg1):
    #Update current round
    current_round[0] += 1
    print("Round ", current_round[0])
    m_rounds.append(Round(current_round[0]))
    
    #Add players to the round
    for play in d.entities.players:
        #In professional matches, there are sometimes players that have no team or are spectators
        if not play.team: continue
        if play.team.name == "Spectator": continue
        m_rounds[-1].add_player(Person(play.name,play.team.name))

def death(event, msg):
    #Stolen bit of code, left in unused features...
    for idx, key in enumerate(event['event'].keys):
        if key.name == 'attacker':
            user_id = msg.keys[idx].val_short
            attacker = d.entities.get_by_user_id(user_id)
        elif key.name == 'userid':
            user_id = msg.keys[idx].val_short
            victim = d.entities.get_by_user_id(user_id)
        elif key.name == 'weapon':
            weapon = msg.keys[idx].val_string
        elif key.name == 'headshot':
            headshot = msg.keys[idx].val_bool
            
    if victim and m_rounds:
        print("Player: ",victim.name," died at tick: ",d.current_tick)
        m_rounds[-1].update_player_position(victim.name,int(victim.position['x']), int(victim.position['y']), victim.view_angle, False,'Y')

In [5]:
#Open and extract DEM file
data = open("deRust.dem", 'rb').read()
d = DemoFile(data)
outFile = open('out.txt', 'w+')

#Add callback events (calls function whenever event occurs in file)
d.add_callback('tick_start', tick_start)
d.add_callback('round_start', round_start)
d.add_callback('player_death', death)

#create list of rounds
m_rounds = []

#Begin parseing file (may take a while)
d.parse()

#Close file
outFile.close()

Round  1
Player:  JWonderchild  died at tick:  6880
Player:  LDLC kiokiNg/w/ * CMSTORM  died at tick:  12416
Player:  KRIMZ  died at tick:  14024
Player:  LDLC shox * CMSTORM  died at tick:  14096
Player:  olofmeister  died at tick:  14112
Player:  pronax  died at tick:  14128
Player:  flusha  died at tick:  14208
Round  2
Player:  pronax  died at tick:  18576
Player:  LDLC kiokiNg/w/ * CMSTORM  died at tick:  26128
Player:  flusha  died at tick:  26344
Player:  LDLC shox * CMSTORM  died at tick:  27024
Player:  LDLC NBKTANK- * CMSTORM  died at tick:  27040
Player:  olofmeister  died at tick:  27096
Player:  LDLC Happy * CMSTORM  died at tick:  28496
Player:  LDLC SMITHZZ * CMSTORM  died at tick:  29040
Round  3
Player:  KRIMZ  died at tick:  38920
Player:  flusha  died at tick:  39288
Player:  olofmeister  died at tick:  43720
Round  4
Player:  KRIMZ  died at tick:  57418
Player:  olofmeister  died at tick:  58738
Player:  pronax  died at tick:  59370
Player:  LDLC kiokiNg/w/ * CMSTOR

Player:  olofmeister  died at tick:  365273


In [6]:
%matplotlib qt

#Print Round 0
m_rounds[0].print_round()

KeyboardInterrupt: 

In [ ]:
#creating seperate class to deal with interesting analysis as modifiying round class 
#requires a full restart to test changes
class round_heat_map:
    def __init__(self):
        self
        
    def print_round (self):
        
    def print_frame (self):
        
    def define_player_info (self, player_info):
        
    